<a href="https://colab.research.google.com/github/Matheus0820/Introducao-a-Otimizacao/blob/main/Problema_da_dieta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Problema da Dieta
- O problema consiste em selecionar os alimentos de uma forma que a minimizar o custo e que o valor nutriconal não seja inferior a certas quantidade e o valor calório seja no máximo igual a **2000kcal**.




Parâmetros:
- $m$: Quantidade de alimentos
- $n$: Quantidade de nutrientes
- $c_i$ : Custo do alimento $i$
- $cal_i$ : Calorias no alimento $i$
- $nut_{ij}$ : Quantidade do nutriente $j$ no alimento $i$
- $\underline{nut}_j$ : Minimo do nutriente j a ser atendido
- $\overline{CAL}$: Total de calorias máximo

Variáveis de decisão:

$$
x_i \in \mathbb{R}_{+} \tag{1}
$$



Função:

$$
\text{min}~Z =  \sum_{j=1}^{n} c_ix_i \tag{2}
$$

Sujeito a:

$$
\sum_{i=1}^{n} cal_ix_i \leq \overline{CAL} \tag{3}
$$

$$
\sum_{i=1}^{m} nut_{ij}x_i \geq \underline{nut}_j,~∀ j=1,...,n
$$

# Configurando o AMPL

In [ ]:
%pip install -q amplpy
from amplpy import AMPL, ampl_notebook
ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi", "cplex"], # pick from over 20 modules including most commercial and open-source solvers
    license_uuid="79c45049-0fee-47f7-a329-ec026ff58c82") # your license UUID (e.g., free ampl.com/ce or ampl.com/courses licenses)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 30.8 MB/s eta 0:00:00
Licensed to AMPL Academic Community Edition License for <matheus.ramos.703@ufrn.edu.br>.


# Arquivo .dat

In [ ]:
%%writefile dieta.dat
param m := 13;
param n := 5;
param CAL_max := 2000;
param : nut_j:=
      1 46
      2 1
      3 1.5
      4 0.018
      5 0.771;

Overwriting dieta.dat


In [ ]:
import pandas as pd

# Lendo os dados do arquivo dados.csv e guardando como um dataframe
df = pd.read_csv('dados.csv')

# Pegando o valor do custo, calorias e nutricionais de cada alimento no dataframe
custo = "param custo := " + "\n".join(f"{i+1} {v}" for i, v in enumerate(df.values[0][1:])) + ";"
calorias = "param cal := " + "\n".join(f"{i+1} {v}" for i, v in enumerate(df.values[1][1:])) + ";"
alimento = "param alimento := " + "\n".join(f'{i+1} "{v}"' for i, v in enumerate(df.columns[1:])) + ";"

nut_linha = list(" ".join(map(str, linha)) for linha in df.values[2:, 1:].T)
nut = "param nut_ij: 1 2 3 4 5 := "
nut += "\n".join(f"{i+1} {v}" for i, v in enumerate(nut_linha)) + " ;"

# Gravando no aquivo dieta.dat
with open('dieta.dat', 'a') as f:
  f.write(alimento + "\n")
  f.write(custo + "\n")
  f.write(calorias + "\n")
  f.write(nut + "\n")
  print("Gravado em dieta.dat")

Gravado em dieta.dat


# Arquivo .mod

In [ ]:
%%writefile dieta.mod
param m >= 0;
param n >= 0;
param CAL_max >= 0;
param custo {1..m} >= 0;
param cal {1..m} >= 0;
param nut_ij {1..m, 1..n} >= 0;
param nut_j {1..n} >= 0;
param alimento {1..m} symbolic;

var x {1..m} >= 0;

minimize Z: sum{i in 1..m} custo[i]*x[i];

subject to calorias: sum{i in 1..m} cal[i]*x[i] <= CAL_max;
subject to nutriente {j in 1..n}: sum{i in 1..m} nut_ij[i, j]*x[i] >= nut_j[j];





Overwriting dieta.mod


# Arquivo .run

In [ ]:
%%writefile dieta.run
reset;
model dieta.mod;
data dieta.dat;
option solver cplex;
solve;
display Z;
printf "Alimentos escolhidos: \n";
for{i in 1..m} {
  if x[i] != 0 then
  {
    printf " %s\n", alimento[i];
  }
}

Overwriting dieta.run


# Execultando a resolução do problema

In [ ]:
%%shell
ampl dieta.run

CPLEX 22.1.2: CPLEX 22.1.2: optimal solution; objective 13.22759549
4 simplex iterations
Z = 13.2276

Alimentos escolhidos: 
 pão integral
 salada crua
 frango grelhado
 iogurte
